本 notebook 对比当各个detector选取自己最大的disagreement点做阈值的时候，是否能够起到如下两个作用：
1. 提高f1
2. 甄别 detector

In [9]:
from __future__ import division,print_function
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import os
import sys
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
import re
from sklearn.preprocessing import RobustScaler
import scipy.stats as ss
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import scipy.io
import os
import sys
from time import time
import scipy.stats as ss
from sklearn.preprocessing import RobustScaler
from sklearn import metrics

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
from numpy import percentile
import matplotlib.pyplot as plt
import matplotlib.font_manager
# Import all models
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA
from pyod.models.lscp import LSCP # ensemble
import multiprocessing as mp
from IPython.display import display
from sklearn.metrics import roc_auc_score
from thresholders import *


data_path = "/Users/kadima/experiment_any/anomaly-detection/datasets/"
def getData(fileName):
    # check fileName:
    files = [x for x in os.listdir(data_path) if x.endswith(".mat")]
    mat = scipy.io.loadmat(data_path+fileName)
    X = mat["X"]
    y = mat["y"]
    return X, y

def get_score_matrix(X, num_detectors):
    return np.zeros([X.shape[0], num_detectors])

def get_perform_matrix(num_thresholders, num_detectors):
    return np.zeros((num_thresholders, num_detectors))


random_state = np.random.RandomState(10)
outliers_fraction = 0.4
# initialize a set of detectors for LSCP
detector_list = [LOF(n_neighbors=5), LOF(n_neighbors=10), LOF(n_neighbors=15),
                 LOF(n_neighbors=20), LOF(n_neighbors=25), LOF(n_neighbors=30),
                 LOF(n_neighbors=35), LOF(n_neighbors=40), LOF(n_neighbors=45),
                 LOF(n_neighbors=50)]
classifiers = {
    'Angle-based Outlier Detector (ABOD)':
        ABOD(contamination=outliers_fraction),
    'Cluster-based Local Outlier Factor (CBLOF)':
        CBLOF(contamination=outliers_fraction,
              check_estimator=False, random_state=random_state),
    'Feature Bagging':
        FeatureBagging(LOF(n_neighbors=35),
                       contamination=outliers_fraction,
                       random_state=random_state),
    'Histogram-base Outlier Detection (HBOS)': HBOS(
        contamination=outliers_fraction),
    'Isolation Forest': IForest(contamination=outliers_fraction,
                                random_state=random_state, n_estimators = 280),
    'K Nearest Neighbors (KNN)': KNN(
        contamination=outliers_fraction),
    'Average KNN': KNN(method='mean',
                       contamination=outliers_fraction),
    'Local Outlier Factor (LOF)':
        LOF(n_neighbors=35, contamination=outliers_fraction),
    'Minimum Covariance Determinant (MCD)': MCD(
        contamination=outliers_fraction, random_state=random_state),
    'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
    'Principal Component Analysis (PCA)': PCA(
        contamination=outliers_fraction, random_state=random_state),
    'Locally Selective Combination (LSCP)': LSCP(
        detector_list, contamination=outliers_fraction,
        random_state=random_state)
}

names = []
# Show all detectors
for i, clf in enumerate(classifiers.keys()):
    names.append(clf)
#     print('Model', i + 1, clf)

# Fit the models with the generated data and 
# compare model performances
def get_result(X, y, classifiers):
    threshold_records = list()
    # create matrix to store the performance
    score_matrix = get_score_matrix(X, len(classifiers.keys()))
    perform_table = get_perform_matrix(6, len(classifiers.keys()) )

    np.random.seed(5)
    clfs = []

    # Fit the model
    for i, (clf_name, clf) in enumerate(classifiers.items()):
#         print(i + 1, 'fitting', clf_name)
        # fit the data and tag outliers
        clf.fit(X)
        clfs.append(clf)
        scores_pred = clf.decision_function(X)
        score_matrix[:, i] = scores_pred
        

    for i, thresholder in enumerate([sd_thresholder, mad_thresholder, 
                                     iqr_thresholder]):
        kk = []
        for j in range(score_matrix.shape[1]):
            _,perform_table[i,j],b = thresholder(score_matrix[:,j], y)
            kk.append(b)
            
        threshold_records.append(kk)
            
    for i in range(score_matrix.shape[1]):
        perform_table[-3, i] = f1_score(y, clfs[i].predict(X))
        perform_table[-2, i] = roc_auc_score(y, clf.predict_proba(X)[:, 1])

        
    perform_table[-1,:], d_threshold, disagreement_record = disagreement_vector(score_matrix, len(classifiers), y, False)
    threshold_records.append(d_threshold)
    
    return (pd.DataFrame(perform_table, columns = names, 
            index = ["sd",'mad','iqr','default-f1','auc','disagreement']), 
            threshold_records, disagreement_record, score_matrix)

def run_helper(datasets):
    X,y = getData(datasets)
    X = X.astype(np.float64)
    return get_result(X, y, classifiers)


result_dict = dict()
bug_datasets = ["letter.mat","speech.mat","cardio.mat","http.mat", "smtp.mat",
               "ecoli.mat","shuttle.mat","seismic-bumps.arff","mammography.mat","cover.mat"]
for datasets in os.listdir("/Users/kadima/experiment_any/anomaly-detection/datasets/"):
    if datasets in bug_datasets:
        continue
    print(datasets)
    result_dict[datasets] = run_helper(datasets)


display(result_dict["lympho.mat"][0])


pima.mat
wine.mat
mnist.mat


/opt/anaconda3/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:647: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


arrhythmia.mat


/opt/anaconda3/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:647: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


pendigits.mat
musk.mat
thyroid.mat


...

/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3702: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3702: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/opt/anaconda3/lib/python3.8/site-packages

satellite.mat
lympho.mat
annthyroid.mat


/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3702: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3702: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/opt/anaconda3/lib/python3.8/site-packages

ionosphere.mat
wbc.mat
glass.mat
satimage-2.mat
vertebral.mat


,Angle-based Outlier Detector (ABOD),Cluster-based Local Outlier Factor (CBLOF),Feature Bagging,Histogram-base Outlier Detection (HBOS),Isolation Forest,K Nearest Neighbors (KNN),Average KNN,Local Outlier Factor (LOF),Minimum Covariance Determinant (MCD),One-class SVM (OCSVM),Principal Component Analysis (PCA),Locally Selective Combination (LSCP)
sd,0.000000,0.769231,0.666667,0.800000,0.909091,0.600000,0.600000,0.666667,0.400000,0.400000,0.666667,0.666667
mad,0.000000,0.077922,0.077922,0.000000,0.833333,0.077922,0.077922,0.077922,0.176471,0.545455,0.077922,0.571429
iqr,0.000000,0.714286,0.750000,0.857143,0.666667,0.500000,0.533333,0.666667,0.400000,0.545455,0.521739,0.545455
default-f1,0.166667,0.184615,0.187500,0.184615,0.184615,0.342857,0.500000,0.184615,0.184615,0.184615,0.184615,0.187500
auc,0.992958,0.994131,0.992958,0.992958,0.992958,0.992958,0.992958,0.994131,0.992958,0.994131,0.994131,0.992958
disagreement,0.077922,0.666667,0.078431,0.193548,0.705882,0.476190,0.727273,0.078431,0.250000,0.631579,0.666667,0.077922


In [11]:
for datasets in os.listdir("/Users/kadima/experiment_any/anomaly-detection/datasets/"):
    if datasets in bug_datasets:
        continue
    else:
        print(datasets)
        print(np.sum(result_dict[datasets][2],axis=0))
        idx_worst = np.argmax(np.median(result_dict[datasets][2],axis=0))
        idx_best = np.argmin(np.median(result_dict[datasets][2],axis=0))

        print(names[idx_worst],'is worst')
        print(names[idx_best], 'is best')
        display(result_dict[datasets][0])
        display(pd.DataFrame(np.asarray(result_dict[datasets][1]).reshape(-1,12),
                             columns=result_dict[datasets][0].columns, index = ['sd', 'mad', 'iqr','disagreement'] ))
        print("=="*55)
        print("\n\n")


pima.mat
[366.65480313  64.61292234  99.72807983  80.59475151  77.76351163
  49.87754318  46.03655995 101.00529601 126.47285652 171.60398828
  69.10970803  98.85645198]
Angle-based Outlier Detector (ABOD) is worst
Average KNN is best


,Angle-based Outlier Detector (ABOD),Cluster-based Local Outlier Factor (CBLOF),Feature Bagging,Histogram-base Outlier Detection (HBOS),Isolation Forest,K Nearest Neighbors (KNN),Average KNN,Local Outlier Factor (LOF),Minimum Covariance Determinant (MCD),One-class SVM (OCSVM),Principal Component Analysis (PCA),Locally Selective Combination (LSCP)
sd,0.000000,0.049645,0.056140,0.095563,0.070175,0.036101,0.043321,0.056140,0.114865,0.00000,0.069204,0.062500
mad,0.000000,0.420650,0.517375,0.517375,0.057143,0.465649,0.472441,0.517375,0.362353,0.00000,0.520737,0.174455
iqr,0.000000,0.108475,0.170732,0.126667,0.070175,0.155340,0.118421,0.171254,0.231638,0.00000,0.127090,0.194529
default-f1,0.481802,0.431304,0.433213,0.528696,0.542609,0.475836,0.379391,0.441652,0.553043,0.42087,0.511304,0.423818
auc,0.566627,0.569396,0.551828,0.568888,0.565925,0.562313,0.572067,0.576858,0.572679,0.56653,0.563903,0.551754
disagreement,0.517375,0.007435,0.021978,0.532020,0.007407,0.007435,0.007435,0.021898,0.007435,0.00000,0.014760,0.021898


,Angle-based Outlier Detector (ABOD),Cluster-based Local Outlier Factor (CBLOF),Feature Bagging,Histogram-base Outlier Detection (HBOS),Isolation Forest,K Nearest Neighbors (KNN),Average KNN,Local Outlier Factor (LOF),Minimum Covariance Determinant (MCD),One-class SVM (OCSVM),Principal Component Analysis (PCA),Locally Selective Combination (LSCP)
sd,0.000039,90.983980,1.598976,23.108828,0.120390,57.780390,38.123400,1.612252,61.396758,0.000733,448.700296,2.369168
mad,0.000008,38.436943,0.234055,2.817009,0.130194,21.504689,15.494152,0.240251,20.258582,0.001121,184.899317,1.154838
iqr,0.000012,68.463070,1.288565,22.778228,0.123751,38.740957,28.037795,1.293413,34.894535,0.001047,408.940278,0.995489
disagreement,-26.940124,18.830786,13.140147,0.100100,3.305545,25.220225,20.106786,12.602830,12.756072,0.936289,4.858126,10.455530





wine.mat
[63.89470459 54.79506831 39.34710159 52.42306554 45.57921572 26.13944612
 21.95048553 43.13710561 22.78725683 48.3040162  37.38148584 41.43984702]
Angle-based Outlier Detector (ABOD) is worst
Average KNN is best


,Angle-based Outlier Detector (ABOD),Cluster-based Local Outlier Factor (CBLOF),Feature Bagging,Histogram-base Outlier Detection (HBOS),Isolation Forest,K Nearest Neighbors (KNN),Average KNN,Local Outlier Factor (LOF),Minimum Covariance Determinant (MCD),One-class SVM (OCSVM),Principal Component Analysis (PCA),Locally Selective Combination (LSCP)
sd,0.000000,0.666667,0.666667,0.000000,0.000000,0.666667,0.750000,0.666667,0.588235,0.000000,0.133333,0.666667
mad,0.000000,0.645161,0.143885,0.143885,0.000000,0.425532,0.400000,0.143885,0.400000,0.095238,0.144928,0.645161
iqr,0.000000,0.833333,0.666667,0.000000,0.000000,0.833333,0.769231,0.645161,0.769231,0.105263,0.125000,0.645161
default-f1,0.333333,0.322581,0.333333,0.258065,0.258065,0.392157,0.526316,0.333333,0.322581,0.096774,0.258065,0.317460
auc,0.997479,0.999160,0.997479,0.997479,0.997479,0.997479,0.997479,0.997479,0.997479,0.997479,0.997479,0.997479
disagreement,0.166667,0.461538,0.273973,0.000000,0.000000,0.461538,0.181818,0.461538,0.166667,0.000000,0.000000,0.461538


...

,Angle-based Outlier Detector (ABOD),Cluster-based Local Outlier Factor (CBLOF),Feature Bagging,Histogram-base Outlier Detection (HBOS),Isolation Forest,K Nearest Neighbors (KNN),Average KNN,Local Outlier Factor (LOF),Minimum Covariance Determinant (MCD),One-class SVM (OCSVM),Principal Component Analysis (PCA),Locally Selective Combination (LSCP)
sd,0.000000,0.788571,0.076336,0.476190,0.605505,0.292035,0.270742,0.079710,0.611354,0.000000,0.730539,0.095588
mad,0.000000,0.082176,0.024174,0.024174,0.382022,0.031640,0.027434,0.024174,0.077469,0.000000,0.051207,0.069606
iqr,0.000000,0.474227,0.069307,0.299020,0.346939,0.203297,0.185792,0.061758,0.193724,0.000000,0.721893,0.070922
default-f1,0.041651,0.059365,0.021515,0.058528,0.059365,0.063120,0.111226,0.021432,0.059365,0.020067,0.058528,0.022594
auc,0.468481,0.495169,0.509581,0.494081,0.508595,0.476389,0.500727,0.475438,0.461970,0.493879,0.473337,0.487459
disagreement,0.000000,0.660377,0.058104,0.434164,0.720721,0.201635,0.301887,0.045403,0.027778,0.000000,0.116554,0.044355


,Angle-based Outlier Detector (ABOD),Cluster-based Local Outlier Factor (CBLOF),Feature Bagging,Histogram-base Outlier Detection (HBOS),Isolation Forest,K Nearest Neighbors (KNN),Average KNN,Local Outlier Factor (LOF),Minimum Covariance Determinant (MCD),One-class SVM (OCSVM),Principal Component Analysis (PCA),Locally Selective Combination (LSCP)
sd,2.482073e-07,98.547270,1.481818,114.278031,0.107033,46.146462,33.522359,1.485320,483.075927,0.000500,398083.189650,2.385965
mad,2.813051e-07,44.385852,0.271691,2.910328,0.080374,18.662439,13.187549,0.268408,54.579015,0.000448,213127.484306,1.703752
iqr,1.756973e-07,78.892108,1.384233,113.270786,0.073991,40.879043,29.914119,1.379072,111.110006,0.000523,396587.318374,1.727466
disagreement,6.556871e-01,7.282107,1.572773,2.623973,5.929176,2.086607,4.029380,1.278273,64.370268,1.770253,0.623461,1.126564





vertebral.mat
[ 75.97571458  18.45854191  29.65438286  48.54304296  20.73048212
  20.30582149  20.58091982  33.32901263  59.34086637 106.38187671
  17.56042875  29.55102668]
One-class SVM (OCSVM) is worst
Cluster-based Local Outlier Factor (CBLOF) is best


,Angle-based Outlier Detector (ABOD),Cluster-based Local Outlier Factor (CBLOF),Feature Bagging,Histogram-base Outlier Detection (HBOS),Isolation Forest,K Nearest Neighbors (KNN),Average KNN,Local Outlier Factor (LOF),Minimum Covariance Determinant (MCD),One-class SVM (OCSVM),Principal Component Analysis (PCA),Locally Selective Combination (LSCP)
sd,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
mad,0.000000,0.088496,0.222222,0.222222,0.000000,0.114286,0.079365,0.222222,0.027397,0.000000,0.122699,0.000000
iqr,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.042553
default-f1,0.125984,0.142857,0.169492,0.111111,0.079365,0.088496,0.025316,0.177419,0.126984,0.269841,0.079365,0.190476
auc,0.479048,0.496508,0.480952,0.488571,0.484444,0.488095,0.481111,0.478571,0.477302,0.470635,0.467778,0.467619
disagreement,0.222222,0.000000,0.000000,0.039216,0.222222,0.000000,0.000000,0.000000,0.000000,0.264706,0.000000,0.000000


,Angle-based Outlier Detector (ABOD),Cluster-based Local Outlier Factor (CBLOF),Feature Bagging,Histogram-base Outlier Detection (HBOS),Isolation Forest,K Nearest Neighbors (KNN),Average KNN,Local Outlier Factor (LOF),Minimum Covariance Determinant (MCD),One-class SVM (OCSVM),Principal Component Analysis (PCA),Locally Selective Combination (LSCP)
sd,0.000040,75.881503,2.226696,19.228757,0.141878,71.473083,53.479981,2.241742,138.602298,0.000629,6.905985e+08,2.380059
mad,0.000012,22.213174,0.258716,0.906859,0.108217,16.775620,12.556720,0.228372,16.113625,0.000608,3.068863e+08,0.571784
iqr,0.000016,40.199184,1.314957,19.236822,0.109210,34.385352,24.838450,1.292260,22.683827,0.000501,6.155898e+08,0.408333
disagreement,-17.365574,29.945735,51.003869,1.310635,-0.780589,35.332999,36.963216,55.821803,97.658720,0.039194,2.447246e+00,49.917895
